# Advent of Code 2019

<https://adventofcode.com/2019/>

In [158]:
import functools
import itertools
import math
import operator
import pprint
import re
import string
import time
import unicodedata

from collections import defaultdict, deque, namedtuple, Counter
from contextlib import contextmanager
from enum import Enum, IntEnum, IntFlag
from fractions import Fraction
from itertools import chain

In [2]:
def pr(x, *extra):
    """Print-Return
    Simple function to aid debugging stuff
    """
    print(repr(x), *extra)
    return x

def debugdecorator(f):
    """Decorator that prints the function call and the returned value"""
    @functools.wraps(f)
    def decorated(*args, **kwargs):
        ret = f(*args, **kwargs)
        print("{}({}) -> {}".format(
            f.__name__,
            ", ".join(chain(
                (repr(a) for a in args),
                ('{}={!r}'.format(k, v) for k,v in sorted(kwargs.items())),
            )),
            repr(ret)
        ))
        return ret
    return decorated

In [3]:
@debugdecorator
def example(a, b, c, d):
    """Sample doc"""
    return (a, b, c, d)

example(1, 2, 3, 4)
example(a=1, b=2, c=3, d=4)
example(1, b=2, d=4, c=3)
print(example.__name__)
print(example.__doc__)

example(1, 2, 3, 4) -> (1, 2, 3, 4)
example(a=1, b=2, c=3, d=4) -> (1, 2, 3, 4)
example(1, b=2, c=3, d=4) -> (1, 2, 3, 4)
example
Sample doc


# Day 1: The Tyranny of the Rocket Equation

<https://adventofcode.com/2019/day/1>

In summary, apply some simple formula to a bunch of numbers.

In [4]:
def day1input():
    with open("day1input.txt") as f:
        for line in f.readlines():
            yield int(line.strip())

def day1fuel(mass):
    return max((mass // 3) - 2, 0)

def day1():
    return sum(day1fuel(x) for x in day1input())

def day1fuelrecursive(mass):
    fuel = day1fuel(mass)
    return fuel + (day1fuelrecursive(fuel) if fuel > 0 else 0)

def day1b_correct():
    # This should have been the correct answer if I was the engineer planning the required fuel.
    f = day1()
    return f + day1fuelrecursive(f)

def day1b_thatsolvesthequestion():
    # This is the expected answer for the problem, it is smaller than the correct one.
    return sum(day1fuelrecursive(x) for x in day1input())

day1(), day1b_correct(), day1b_thatsolvesthequestion()

(3372463, 5058654, 5055835)

# Day 2: 1202 Program Alarm

<https://adventofcode.com/2019/day/2>

Simple CPU emulator.

In [5]:
def day2input():
    with open("day2input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

day2Instuction = namedtuple("day2Instruction", "opcode name length")

class day2Exception(Exception):
    pass

class day2Computer:
    instructions = {
        i.opcode: i
        for i in [
            day2Instuction(1, "add", 4),
            day2Instuction(2, "mul", 4),
            day2Instuction(99, "halt", 1),
        ]
    }

    def __init__(self, mem):
        self.mem = mem
        self.pc = 0  # Program Counter, AKA Instruction Pointer
        self.halted = False

    def __repr__(self):
        return '<day2Computer: mem_length={}, pc={}, halted={}>'.format(
            len(self.mem),
            self.pc,
            self.halted
        )

    def step(self):
        """Executes one instruction"""
        opcode = self.mem[self.pc]
        
        if opcode not in self.instructions:
            raise day2Exception("Invalid opcode {} at position {}".format(opcode, self.pc))

        instruction = self.instructions[opcode]
        args = self.mem[self.pc:self.pc+instruction.length]
        #print("pc={}: {} {}".format(self.pc, instruction, args))
        self.pc += instruction.length
        getattr(self, 'execute_' + instruction.name)(*args)

    def run(self, limit=None):
        while not self.halted and (limit is None or limit > 0):
            if limit is not None:
                limit -=1
            self.step()

    def execute_add(self, opcode, a, b, dst):
        self.mem[dst] = self.mem[a] + self.mem[b]
    def execute_mul(self, opcode, a, b, dst):
        self.mem[dst] = self.mem[a] * self.mem[b]
    def execute_halt(self, opcode):
        self.halted = True


def day2sample():
    mem = [1,9,10,3,2,3,11,0,99,30,40,50]
    computer = day2Computer(mem)
    computer.run(limit=9999)
    print(computer)
    print(mem)

def day2():
    mem = day2input()
    # before running the program, replace position 1 with the value 12 and replace position 2 with the value 2
    mem[1] = 12
    mem[2] = 2
    computer = day2Computer(mem)
    computer.run(limit=9999)
    print(computer)
    print(mem)

day2sample()
day2()

<day2Computer: mem_length=12, pc=9, halted=True>
[3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]
<day2Computer: mem_length=193, pc=189, halted=True>
[4023471, 12, 2, 2, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 1, 6, 24, 1, 5, 19, 25, 1, 23, 6, 27, 1, 5, 27, 28, 1, 31, 6, 30, 1, 9, 35, 33, 2, 10, 39, 132, 1, 43, 6, 134, 2, 6, 47, 268, 1, 5, 51, 269, 1, 55, 13, 274, 1, 59, 10, 278, 2, 10, 63, 1112, 1, 9, 67, 1115, 2, 6, 71, 2230, 1, 5, 75, 2231, 2, 79, 13, 11155, 1, 83, 5, 11156, 1, 87, 9, 11159, 1, 5, 91, 11160, 1, 5, 95, 11161, 1, 99, 13, 11166, 1, 10, 103, 11170, 1, 107, 9, 11173, 1, 6, 111, 11175, 2, 115, 13, 55875, 1, 10, 119, 55879, 2, 123, 6, 111758, 1, 5, 127, 111759, 1, 5, 131, 111760, 1, 135, 6, 111762, 2, 139, 10, 447048, 2, 143, 9, 1341144, 1, 147, 6, 1341146, 1, 151, 13, 1341151, 2, 155, 9, 4023453, 1, 6, 159, 4023455, 1, 5, 163, 4023456, 1, 5, 167, 4023457, 1, 10, 171, 4023461, 1, 13, 175, 4023466, 1, 179, 2, 4023468, 1, 9, 183, 0, 99, 2, 14, 0, 0]


In [6]:
def day2run_with(noun, verb):
    mem = day2input()
    mem[1] = noun
    mem[2] = verb
    computer = day2Computer(mem)
    computer.run(limit=9999)
    return mem[0]

def day2b():
    for x in range(100):
        for y in range(100):
            ret = day2run_with(x, y)
            if ret == 19690720:
                return x * 100 + y

day2b()

8051

# Day 3: Crossed Wires

<https://adventofcode.com/2019/day/3>

The input describes two wires, and asks to find the crossing closest to the origin.

However, the problem is not clear (or not specific enough), and doesn't say if the wires could ever overlap parallel to each other (because, if this happens, should this be considered a crossing?).

So I decided to implement something to print the lines in a very clear way (using Unicode Box Drawing characters). Unfortunately, it's not as straightforward as I expected (I couldn't find any mapping from the lines to bits; for comparison, Braille characters can be easily generated). It took quite some time to mark all characters to the appropriate bits in the enumerations.

Talking about the enumerations, each direction is a bit, in the same order as CSS (up, right, bottom, left). Bits can be combined and they map to box-drawing characters. Additionally, since we have two wires per test-case, I mapped the lower 4 bits to the first wire, and the upper 4 bits to the second wire; and the first wire is drawn with light lines and the second wire with heavy lines.

Overall, this was a nice thing to be implemented, good for debugging and exploring, but a bit useless and a bit wasteful. So, I'm keeping it here, but I won't use it in the final solution

In [7]:
day3samples = [
    ["R8,U5,L5,D3", "U7,R6,D4,L4"],
    ["R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83"],
    ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"],
]

def day3input():
    with open("day3input.txt") as f:
        return [line.strip() for line in f.read().strip().splitlines()]

def day3parse(line):
    return [
        (segment[0], int(segment[1:]))
        for segment in line.split(',')
    ]

max(x[1] for x in chain(*[day3parse(line) for line in day3input()]))

997

In [8]:
class DIR(IntEnum):
    """Directions and their bit position"""
    n = 0
    e = 1
    s = 2
    w = 3
    N = 4
    E = 5
    S = 6
    W = 7
    up = 0
    right = 1
    down = 2
    left = 3
    UP = 4
    RIGHT = 5
    DOWN = 6
    LEFT = 7
    u = 0
    r = 1
    d = 2
    l = 3
    U = 4
    R = 5
    D = 6
    L = 7

class DIRBIT(IntFlag):
    """Bits for each direction"""
    n = 1 << DIR.n
    e = 1 << DIR.e
    s = 1 << DIR.s
    w = 1 << DIR.w
    N = 1 << DIR.N
    E = 1 << DIR.E
    S = 1 << DIR.S
    W = 1 << DIR.W
    up = 1 << DIR.up
    right = 1 << DIR.right
    down = 1 << DIR.down
    left = 1 << DIR.left
    UP = 1 << DIR.UP
    RIGHT = 1 << DIR.RIGHT
    DOWN = 1 << DIR.DOWN
    LEFT = 1 << DIR.LEFT
    u = 1 << DIR.u
    r = 1 << DIR.r
    d = 1 << DIR.d
    l = 1 << DIR.l
    U = 1 << DIR.U
    R = 1 << DIR.R
    D = 1 << DIR.D
    L = 1 << DIR.L

DirChar = namedtuple("DirChar", "char dirs name")

In [9]:
def print_box_chars():
    box_chars = ''.join(chr(x) for x in range(0x2500, 0x2570))
    print(box_chars)
    for c in box_chars:
        print("DirChar({!r}, DIRBIT.u|DIRBIT.d, {!r}),".format(
            c,
            hex(ord(c)) + " " + unicodedata.name(c),
        ))

print_box_chars()

─━│┃┄┅┆┇┈┉┊┋┌┍┎┏┐┑┒┓└┕┖┗┘┙┚┛├┝┞┟┠┡┢┣┤┥┦┧┨┩┪┫┬┭┮┯┰┱┲┳┴┵┶┷┸┹┺┻┼┽┾┿╀╁╂╃╄╅╆╇╈╉╊╋╌╍╎╏═║╒╓╔╕╖╗╘╙╚╛╜╝╞╟╠╡╢╣╤╥╦╧╨╩╪╫╬╭╮╯
DirChar('─', DIRBIT.u|DIRBIT.d, '0x2500 BOX DRAWINGS LIGHT HORIZONTAL'),
DirChar('━', DIRBIT.u|DIRBIT.d, '0x2501 BOX DRAWINGS HEAVY HORIZONTAL'),
DirChar('│', DIRBIT.u|DIRBIT.d, '0x2502 BOX DRAWINGS LIGHT VERTICAL'),
DirChar('┃', DIRBIT.u|DIRBIT.d, '0x2503 BOX DRAWINGS HEAVY VERTICAL'),
DirChar('┄', DIRBIT.u|DIRBIT.d, '0x2504 BOX DRAWINGS LIGHT TRIPLE DASH HORIZONTAL'),
DirChar('┅', DIRBIT.u|DIRBIT.d, '0x2505 BOX DRAWINGS HEAVY TRIPLE DASH HORIZONTAL'),
DirChar('┆', DIRBIT.u|DIRBIT.d, '0x2506 BOX DRAWINGS LIGHT TRIPLE DASH VERTICAL'),
DirChar('┇', DIRBIT.u|DIRBIT.d, '0x2507 BOX DRAWINGS HEAVY TRIPLE DASH VERTICAL'),
DirChar('┈', DIRBIT.u|DIRBIT.d, '0x2508 BOX DRAWINGS LIGHT QUADRUPLE DASH HORIZONTAL'),
DirChar('┉', DIRBIT.u|DIRBIT.d, '0x2509 BOX DRAWINGS HEAVY QUADRUPLE DASH HORIZONTAL'),
DirChar('┊', DIRBIT.u|DIRBIT.d, '0x250a BOX DRAWINGS LIGHT QUADRUPLE DASH VERTICAL'),
D

In [10]:
BoxDrawingChars = {
    c.dirs: c
    for c in [
        DirChar("─", DIRBIT.r|DIRBIT.l, '0x2500 BOX DRAWINGS LIGHT HORIZONTAL'),
        DirChar("━", DIRBIT.R|DIRBIT.L, '0x2501 BOX DRAWINGS HEAVY HORIZONTAL'),
        DirChar("│", DIRBIT.u|DIRBIT.d, '0x2502 BOX DRAWINGS LIGHT VERTICAL'),
        DirChar("┃", DIRBIT.U|DIRBIT.D, '0x2503 BOX DRAWINGS HEAVY VERTICAL'),

        DirChar("┌", DIRBIT.r|DIRBIT.d, '0x250c BOX DRAWINGS LIGHT DOWN AND RIGHT'),
        DirChar("┍", DIRBIT.R|DIRBIT.d, '0x250d BOX DRAWINGS DOWN LIGHT AND RIGHT HEAVY'),
        DirChar("┎", DIRBIT.r|DIRBIT.D, '0x250e BOX DRAWINGS DOWN HEAVY AND RIGHT LIGHT'),
        DirChar("┏", DIRBIT.R|DIRBIT.D, '0x250f BOX DRAWINGS HEAVY DOWN AND RIGHT'),

        DirChar("┐", DIRBIT.d|DIRBIT.l, '0x2510 BOX DRAWINGS LIGHT DOWN AND LEFT'),
        DirChar("┑", DIRBIT.d|DIRBIT.L, '0x2511 BOX DRAWINGS DOWN LIGHT AND LEFT HEAVY'),
        DirChar("┒", DIRBIT.D|DIRBIT.l, '0x2512 BOX DRAWINGS DOWN HEAVY AND LEFT LIGHT'),
        DirChar("┓", DIRBIT.D|DIRBIT.L, '0x2513 BOX DRAWINGS HEAVY DOWN AND LEFT'),

        DirChar("└", DIRBIT.u|DIRBIT.r, '0x2514 BOX DRAWINGS LIGHT UP AND RIGHT'),
        DirChar("┕", DIRBIT.u|DIRBIT.R, '0x2515 BOX DRAWINGS UP LIGHT AND RIGHT HEAVY'),
        DirChar("┖", DIRBIT.U|DIRBIT.r, '0x2516 BOX DRAWINGS UP HEAVY AND RIGHT LIGHT'),
        DirChar("┗", DIRBIT.U|DIRBIT.R, '0x2517 BOX DRAWINGS HEAVY UP AND RIGHT'),

        DirChar("┘", DIRBIT.u|DIRBIT.l, '0x2518 BOX DRAWINGS LIGHT UP AND LEFT'),
        DirChar("┙", DIRBIT.u|DIRBIT.L, '0x2519 BOX DRAWINGS UP LIGHT AND LEFT HEAVY'),
        DirChar("┚", DIRBIT.U|DIRBIT.l, '0x251a BOX DRAWINGS UP HEAVY AND LEFT LIGHT'),
        DirChar("┛", DIRBIT.U|DIRBIT.L, '0x251b BOX DRAWINGS HEAVY UP AND LEFT'),

        DirChar("├", DIRBIT.u|DIRBIT.r|DIRBIT.d, '0x251c BOX DRAWINGS LIGHT VERTICAL AND RIGHT'),
        DirChar("┝", DIRBIT.u|DIRBIT.R|DIRBIT.d, '0x251d BOX DRAWINGS VERTICAL LIGHT AND RIGHT HEAVY'),
        DirChar("┞", DIRBIT.U|DIRBIT.r|DIRBIT.d, '0x251e BOX DRAWINGS UP HEAVY AND RIGHT DOWN LIGHT'),
        DirChar("┟", DIRBIT.u|DIRBIT.r|DIRBIT.D, '0x251f BOX DRAWINGS DOWN HEAVY AND RIGHT UP LIGHT'),
        DirChar("┠", DIRBIT.U|DIRBIT.r|DIRBIT.D, '0x2520 BOX DRAWINGS VERTICAL HEAVY AND RIGHT LIGHT'),
        DirChar("┡", DIRBIT.U|DIRBIT.R|DIRBIT.d, '0x2521 BOX DRAWINGS DOWN LIGHT AND RIGHT UP HEAVY'),
        DirChar("┢", DIRBIT.u|DIRBIT.R|DIRBIT.D, '0x2522 BOX DRAWINGS UP LIGHT AND RIGHT DOWN HEAVY'),
        DirChar("┣", DIRBIT.U|DIRBIT.R|DIRBIT.D, '0x2523 BOX DRAWINGS HEAVY VERTICAL AND RIGHT'),

        DirChar("┤", DIRBIT.u|DIRBIT.d|DIRBIT.l, '0x2524 BOX DRAWINGS LIGHT VERTICAL AND LEFT'),
        DirChar("┥", DIRBIT.u|DIRBIT.d|DIRBIT.L, '0x2525 BOX DRAWINGS VERTICAL LIGHT AND LEFT HEAVY'),
        DirChar("┦", DIRBIT.U|DIRBIT.d|DIRBIT.l, '0x2526 BOX DRAWINGS UP HEAVY AND LEFT DOWN LIGHT'),
        DirChar("┧", DIRBIT.u|DIRBIT.D|DIRBIT.l, '0x2527 BOX DRAWINGS DOWN HEAVY AND LEFT UP LIGHT'),
        DirChar("┨", DIRBIT.U|DIRBIT.D|DIRBIT.l, '0x2528 BOX DRAWINGS VERTICAL HEAVY AND LEFT LIGHT'),
        DirChar("┩", DIRBIT.U|DIRBIT.d|DIRBIT.L, '0x2529 BOX DRAWINGS DOWN LIGHT AND LEFT UP HEAVY'),
        DirChar("┪", DIRBIT.u|DIRBIT.D|DIRBIT.L, '0x252a BOX DRAWINGS UP LIGHT AND LEFT DOWN HEAVY'),
        DirChar("┫", DIRBIT.U|DIRBIT.D|DIRBIT.L, '0x252b BOX DRAWINGS HEAVY VERTICAL AND LEFT'),

        DirChar("┬", DIRBIT.r|DIRBIT.d|DIRBIT.l, '0x252c BOX DRAWINGS LIGHT DOWN AND HORIZONTAL'),
        DirChar("┭", DIRBIT.r|DIRBIT.d|DIRBIT.L, '0x252d BOX DRAWINGS LEFT HEAVY AND RIGHT DOWN LIGHT'),
        DirChar("┮", DIRBIT.R|DIRBIT.d|DIRBIT.l, '0x252e BOX DRAWINGS RIGHT HEAVY AND LEFT DOWN LIGHT'),
        DirChar("┯", DIRBIT.R|DIRBIT.d|DIRBIT.L, '0x252f BOX DRAWINGS DOWN LIGHT AND HORIZONTAL HEAVY'),
        DirChar("┰", DIRBIT.r|DIRBIT.D|DIRBIT.l, '0x2530 BOX DRAWINGS DOWN HEAVY AND HORIZONTAL LIGHT'),
        DirChar("┱", DIRBIT.r|DIRBIT.D|DIRBIT.L, '0x2531 BOX DRAWINGS RIGHT LIGHT AND LEFT DOWN HEAVY'),
        DirChar("┲", DIRBIT.R|DIRBIT.D|DIRBIT.l, '0x2532 BOX DRAWINGS LEFT LIGHT AND RIGHT DOWN HEAVY'),
        DirChar("┳", DIRBIT.R|DIRBIT.D|DIRBIT.L, '0x2533 BOX DRAWINGS HEAVY DOWN AND HORIZONTAL'),

        DirChar("┴", DIRBIT.u|DIRBIT.r|DIRBIT.l, '0x2534 BOX DRAWINGS LIGHT UP AND HORIZONTAL'),
        DirChar("┵", DIRBIT.u|DIRBIT.r|DIRBIT.L, '0x2535 BOX DRAWINGS LEFT HEAVY AND RIGHT UP LIGHT'),
        DirChar("┶", DIRBIT.u|DIRBIT.R|DIRBIT.l, '0x2536 BOX DRAWINGS RIGHT HEAVY AND LEFT UP LIGHT'),
        DirChar("┷", DIRBIT.u|DIRBIT.R|DIRBIT.L, '0x2537 BOX DRAWINGS UP LIGHT AND HORIZONTAL HEAVY'),
        DirChar("┸", DIRBIT.U|DIRBIT.r|DIRBIT.l, '0x2538 BOX DRAWINGS UP HEAVY AND HORIZONTAL LIGHT'),
        DirChar("┹", DIRBIT.U|DIRBIT.r|DIRBIT.L, '0x2539 BOX DRAWINGS RIGHT LIGHT AND LEFT UP HEAVY'),
        DirChar("┺", DIRBIT.U|DIRBIT.R|DIRBIT.l, '0x253a BOX DRAWINGS LEFT LIGHT AND RIGHT UP HEAVY'),
        DirChar("┻", DIRBIT.U|DIRBIT.R|DIRBIT.L, '0x253b BOX DRAWINGS HEAVY UP AND HORIZONTAL'),

        DirChar("┼", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.l, '0x253c BOX DRAWINGS LIGHT VERTICAL AND HORIZONTAL'),
        DirChar("┽", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.L, '0x253d BOX DRAWINGS LEFT HEAVY AND RIGHT VERTICAL LIGHT'),
        DirChar("┾", DIRBIT.u|DIRBIT.R|DIRBIT.d|DIRBIT.l, '0x253e BOX DRAWINGS RIGHT HEAVY AND LEFT VERTICAL LIGHT'),
        DirChar("┿", DIRBIT.u|DIRBIT.R|DIRBIT.d|DIRBIT.L, '0x253f BOX DRAWINGS VERTICAL LIGHT AND HORIZONTAL HEAVY'),
        DirChar("╀", DIRBIT.U|DIRBIT.r|DIRBIT.d|DIRBIT.l, '0x2540 BOX DRAWINGS UP HEAVY AND DOWN HORIZONTAL LIGHT'),
        DirChar("╁", DIRBIT.u|DIRBIT.r|DIRBIT.D|DIRBIT.l, '0x2541 BOX DRAWINGS DOWN HEAVY AND UP HORIZONTAL LIGHT'),
        DirChar("╂", DIRBIT.U|DIRBIT.r|DIRBIT.D|DIRBIT.l, '0x2542 BOX DRAWINGS VERTICAL HEAVY AND HORIZONTAL LIGHT'),
        DirChar("╃", DIRBIT.U|DIRBIT.r|DIRBIT.d|DIRBIT.L, '0x2543 BOX DRAWINGS LEFT UP HEAVY AND RIGHT DOWN LIGHT'),
        DirChar("╄", DIRBIT.U|DIRBIT.R|DIRBIT.d|DIRBIT.l, '0x2544 BOX DRAWINGS RIGHT UP HEAVY AND LEFT DOWN LIGHT'),
        DirChar("╅", DIRBIT.u|DIRBIT.r|DIRBIT.D|DIRBIT.L, '0x2545 BOX DRAWINGS LEFT DOWN HEAVY AND RIGHT UP LIGHT'),
        DirChar("╆", DIRBIT.u|DIRBIT.R|DIRBIT.D|DIRBIT.l, '0x2546 BOX DRAWINGS RIGHT DOWN HEAVY AND LEFT UP LIGHT'),
        DirChar("╇", DIRBIT.U|DIRBIT.R|DIRBIT.d|DIRBIT.L, '0x2547 BOX DRAWINGS DOWN LIGHT AND UP HORIZONTAL HEAVY'),
        DirChar("╈", DIRBIT.u|DIRBIT.R|DIRBIT.D|DIRBIT.L, '0x2548 BOX DRAWINGS UP LIGHT AND DOWN HORIZONTAL HEAVY'),
        DirChar("╉", DIRBIT.U|DIRBIT.r|DIRBIT.D|DIRBIT.L, '0x2549 BOX DRAWINGS RIGHT LIGHT AND LEFT VERTICAL HEAVY'),
        DirChar("╊", DIRBIT.U|DIRBIT.R|DIRBIT.D|DIRBIT.l, '0x254a BOX DRAWINGS LEFT LIGHT AND RIGHT VERTICAL HEAVY'),
        DirChar("╋", DIRBIT.U|DIRBIT.R|DIRBIT.D|DIRBIT.L, '0x254b BOX DRAWINGS HEAVY VERTICAL AND HORIZONTAL'),

        DirChar("═", DIRBIT.r|DIRBIT.l|DIRBIT.R|DIRBIT.L, '0x2550 BOX DRAWINGS DOUBLE HORIZONTAL'),
        DirChar("║", DIRBIT.u|DIRBIT.d|DIRBIT.U|DIRBIT.D, '0x2551 BOX DRAWINGS DOUBLE VERTICAL'),

        DirChar("╒", DIRBIT.r|DIRBIT.d|DIRBIT.R, '0x2552 BOX DRAWINGS DOWN SINGLE AND RIGHT DOUBLE'),
        DirChar("╓", DIRBIT.r|DIRBIT.d|DIRBIT.D, '0x2553 BOX DRAWINGS DOWN DOUBLE AND RIGHT SINGLE'),
        DirChar("╔", DIRBIT.r|DIRBIT.d|DIRBIT.R|DIRBIT.D, '0x2554 BOX DRAWINGS DOUBLE DOWN AND RIGHT'),
        DirChar("╕", DIRBIT.d|DIRBIT.l|DIRBIT.L, '0x2555 BOX DRAWINGS DOWN SINGLE AND LEFT DOUBLE'),
        DirChar("╖", DIRBIT.d|DIRBIT.l|DIRBIT.D, '0x2556 BOX DRAWINGS DOWN DOUBLE AND LEFT SINGLE'),
        DirChar("╗", DIRBIT.d|DIRBIT.l|DIRBIT.D|DIRBIT.L, '0x2557 BOX DRAWINGS DOUBLE DOWN AND LEFT'),
        DirChar("╘", DIRBIT.u|DIRBIT.r|DIRBIT.R, '0x2558 BOX DRAWINGS UP SINGLE AND RIGHT DOUBLE'),
        DirChar("╙", DIRBIT.u|DIRBIT.r|DIRBIT.U, '0x2559 BOX DRAWINGS UP DOUBLE AND RIGHT SINGLE'),
        DirChar("╚", DIRBIT.u|DIRBIT.r|DIRBIT.U|DIRBIT.R, '0x255a BOX DRAWINGS DOUBLE UP AND RIGHT'),
        DirChar("╛", DIRBIT.u|DIRBIT.l|DIRBIT.L, '0x255b BOX DRAWINGS UP SINGLE AND LEFT DOUBLE'),
        DirChar("╜", DIRBIT.u|DIRBIT.l|DIRBIT.U, '0x255c BOX DRAWINGS UP DOUBLE AND LEFT SINGLE'),
        DirChar("╝", DIRBIT.u|DIRBIT.l|DIRBIT.U|DIRBIT.L, '0x255d BOX DRAWINGS DOUBLE UP AND LEFT'),

        DirChar("╞", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.R, '0x255e BOX DRAWINGS VERTICAL SINGLE AND RIGHT DOUBLE'),
        DirChar("╟", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.U|DIRBIT.D, '0x255f BOX DRAWINGS VERTICAL DOUBLE AND RIGHT SINGLE'),
        DirChar("╠", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.U|DIRBIT.R|DIRBIT.D, '0x2560 BOX DRAWINGS DOUBLE VERTICAL AND RIGHT'),
        DirChar("╡", DIRBIT.u|DIRBIT.d|DIRBIT.l|DIRBIT.L, '0x2561 BOX DRAWINGS VERTICAL SINGLE AND LEFT DOUBLE'),
        DirChar("╢", DIRBIT.u|DIRBIT.d|DIRBIT.l|DIRBIT.U|DIRBIT.D, '0x2562 BOX DRAWINGS VERTICAL DOUBLE AND LEFT SINGLE'),
        DirChar("╣", DIRBIT.u|DIRBIT.d|DIRBIT.l|DIRBIT.U|DIRBIT.D|DIRBIT.L, '0x2563 BOX DRAWINGS DOUBLE VERTICAL AND LEFT'),
        DirChar("╤", DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.R|DIRBIT.L, '0x2564 BOX DRAWINGS DOWN SINGLE AND HORIZONTAL DOUBLE'),
        DirChar("╥", DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.D, '0x2565 BOX DRAWINGS DOWN DOUBLE AND HORIZONTAL SINGLE'),
        DirChar("╦", DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.R|DIRBIT.D|DIRBIT.L, '0x2566 BOX DRAWINGS DOUBLE DOWN AND HORIZONTAL'),
        DirChar("╧", DIRBIT.u|DIRBIT.r|DIRBIT.l|DIRBIT.R|DIRBIT.L, '0x2567 BOX DRAWINGS UP SINGLE AND HORIZONTAL DOUBLE'),
        DirChar("╨", DIRBIT.u|DIRBIT.r|DIRBIT.l|DIRBIT.U, '0x2568 BOX DRAWINGS UP DOUBLE AND HORIZONTAL SINGLE'),
        DirChar("╩", DIRBIT.u|DIRBIT.r|DIRBIT.l|DIRBIT.U|DIRBIT.R|DIRBIT.L, '0x2569 BOX DRAWINGS DOUBLE UP AND HORIZONTAL'),

        DirChar("╪", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.R|DIRBIT.L, '0x256a BOX DRAWINGS VERTICAL SINGLE AND HORIZONTAL DOUBLE'),
        DirChar("╫", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.U|DIRBIT.D, '0x256b BOX DRAWINGS VERTICAL DOUBLE AND HORIZONTAL SINGLE'),
        DirChar("╬", DIRBIT.u|DIRBIT.r|DIRBIT.d|DIRBIT.l|DIRBIT.U|DIRBIT.R|DIRBIT.D|DIRBIT.L, '0x256c BOX DRAWINGS DOUBLE VERTICAL AND HORIZONTAL'),
    ]
}

def BoxDraw(flags):
    if flags in BoxDrawingChars:
        return BoxDrawingChars[flags].char
    else:
        # Placeholder: Braille patterns
        return chr(0x2800 + flags)

In [11]:
def print_debug_box_chars_from_dirs():
    for x in range(0, 0x100):
        print("{:08b} {}".format(x, BoxDraw(x)))

print_debug_box_chars_from_dirs()

00000000 ⠀
00000001 ⠁
00000010 ⠂
00000011 └
00000100 ⠄
00000101 │
00000110 ┌
00000111 ├
00001000 ⠈
00001001 ┘
00001010 ─
00001011 ┴
00001100 ┐
00001101 ┤
00001110 ┬
00001111 ┼
00010000 ⠐
00010001 ⠑
00010010 ┖
00010011 ╙
00010100 ⠔
00010101 ⠕
00010110 ┞
00010111 ⠗
00011000 ┚
00011001 ╜
00011010 ┸
00011011 ╨
00011100 ┦
00011101 ⠝
00011110 ╀
00011111 ⠟
00100000 ⠠
00100001 ┕
00100010 ⠢
00100011 ╘
00100100 ┍
00100101 ┝
00100110 ╒
00100111 ╞
00101000 ⠨
00101001 ┶
00101010 ⠪
00101011 ⠫
00101100 ┮
00101101 ┾
00101110 ⠮
00101111 ⠯
00110000 ┗
00110001 ⠱
00110010 ⠲
00110011 ╚
00110100 ┡
00110101 ⠵
00110110 ⠶
00110111 ⠷
00111000 ┺
00111001 ⠹
00111010 ⠺
00111011 ⠻
00111100 ╄
00111101 ⠽
00111110 ⠾
00111111 ⠿
01000000 ⡀
01000001 ⡁
01000010 ┎
01000011 ┟
01000100 ⡄
01000101 ⡅
01000110 ╓
01000111 ⡇
01001000 ┒
01001001 ┧
01001010 ┰
01001011 ╁
01001100 ╖
01001101 ⡍
01001110 ╥
01001111 ⡏
01010000 ┃
01010001 ⡑
01010010 ┠
01010011 ⡓
01010100 ⡔
01010101 ║
01010110 ⡖
01010111 ╟
01011000 ┨
01011001 ⡙
01011010 ╂

In [12]:
def day3naiveprint(canvas):
    for row in reversed(canvas):
        print(''.join(
            BoxDraw(flags) for flags in row
        ))

def day3calc_size_for_line(line):
    deltas = {
        "U": ( 0,  1),
        "D": ( 0, -1),
        "L": (-1,  0),
        "R": ( 1,  0),
    }
    x, y = 0, 0
    minx, miny = 0, 0
    maxx, maxy = 0, 0
    for d, length in day3parse(line):
        dx, dy = deltas[d]
        x, y = x + dx * length, y + dy * length
        minx = min(x, minx)
        miny = min(y, miny)
        maxx = max(x, maxx)
        maxy = max(y, maxy)
    return (minx, miny, maxx, maxy)

def day3naive():
    deltas = {
        "U": ( 0,  1, DIRBIT.u, DIRBIT.d),
        "D": ( 0, -1, DIRBIT.d, DIRBIT.u),
        "L": (-1,  0, DIRBIT.l, DIRBIT.r),
        "R": ( 1,  0, DIRBIT.r, DIRBIT.l),
    }
    for lines in day3samples:
        minx, miny, maxx, maxy = 0, 0, 0, 0
        for line in lines:
            a, b, c, d = day3calc_size_for_line(line)
            minx = min(a, minx)
            miny = min(b, miny)
            maxx = max(c, maxx)
            maxy = max(d, maxy)

        canvas = [
            [0] * (maxx - minx + 1)
            for _ in range(maxy - miny + 1)
        ]
        for lineno, line in enumerate(lines):
            x, y = -minx, -miny
            for d, length in day3parse(line):
                print(lineno, x, y, d, length)
                dx, dy, bitsrc, bitdst = deltas[d]
                if lineno == 1:
                    bitsrc = bitsrc << 4
                    bitdst = bitdst << 4
                for i in range(length):
                    nx, ny = x + dx, y + dy
                    canvas[ y][ x] |= bitsrc
                    canvas[ny][nx] |= bitdst
                    x, y = nx, ny
        day3naiveprint(canvas)

day3naive()

0 0 0 R 8
0 8 0 U 5
0 8 5 L 5
0 3 5 D 3
1 0 0 U 7
1 0 7 R 6
1 6 7 D 4
1 6 3 L 4
┏━━━━━┓⠀⠀
┃⠀⠀⠀⠀⠀┃⠀⠀
┃⠀⠀┌──╂─┐
┃⠀⠀│⠀⠀┃⠀│
┃⠀⠠┿━━┛⠀│
┃⠀⠀⠁⠀⠀⠀⠀│
┃⠀⠀⠀⠀⠀⠀⠀│
┖───────┘
0 0 30 R 75
0 75 30 D 30
0 75 0 R 83
0 158 0 U 83
0 158 83 L 12
0 146 83 D 49
0 146 34 R 71
0 217 34 U 7
0 217 41 L 72
1 0 30 U 62
1 0 92 R 66
1 66 92 U 55
1 66 147 R 34
1 100 147 D 71
1 100 76 R 55
1 155 76 D 58
1 155 18 R 83
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀┃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀┃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀┃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀┃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [13]:
class day3Point:
    def __init__(self, x, y, distance=0):
        self.x = x
        self.y = y
        self.distance = distance

    def __repr__(self):
        return 'day3Point({}, {}, {})'.format(self.x, self.y, self.distance)

    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)

    def __hash__(self):
        return hash((self.x, self.y))

def day3get_all_points(line):
    deltas = {
        "U": ( 0,  1),
        "D": ( 0, -1),
        "L": (-1,  0),
        "R": ( 1,  0),
    }
    x, y = 0, 0
    steps = 0
    points = {}
    points[(x, y)] = day3Point(x, y, steps)
    for d, length in day3parse(line):
        dx, dy = deltas[d]
        for i in range(length):
            x, y = x + dx, y + dy
            steps += 1
            if (x, y) not in points:
                points[(x, y)] = day3Point(x, y, steps)
    return points

def day3():
    points = [
        day3get_all_points(line)
        for line in day3input()
    ]
    # Note: To generalize this to more than 2 sets, I could use `reduce`.
    crossings = set(points[0].keys()) & set(points[1].keys())
    closest = min(
        abs(p[0]) + abs(p[1])  # Manhattan distance
        for p in crossings
        if p[0] != 0 and p[1] != 0  # Ignoring the origin.
    )
    return closest


def day3b():
    points = [
        day3get_all_points(line)
        for line in day3input()
    ]
    distances = [
        a.distance + points[1][(a.x, a.y)].distance
        for a in points[0].values()
        if (a.x, a.y) in points[1] and (a.x, a.y) != (0, 0)
    ]
    return min(distances)

day3(), day3b()

(232, 6084)

# Day 4: Secure Container

<https://adventofcode.com/2019/day/4>

Counting how many numbers (passwords) matches a few arbitrary rules.

In [14]:
def day4bruteforce():
    for x in range(168630, 718098 + 1):
        s = str(x)
        double_digits = any(s[i] == s[i+1] for i in range(len(s) - 1))
        non_decreasing = all(s[i] <= s[i+1] for i in range(len(s) - 1))
        if double_digits and non_decreasing:
            yield x

def day4count_repeating(input):
    it = iter(input)
    prev = next(it)
    count = 1
    for item in it:
        if item == prev:
            count += 1
        else:
            yield (prev, count)
            prev = item
            count = 1
    yield (prev, count)        

def day4b_bruteforce():
    for x in range(168630, 718098 + 1):
        s = str(x)
        repeating = day4count_repeating(s)
        double_digits = any(x[1] == 2 for x in repeating)
        non_decreasing = all(s[i] <= s[i+1] for i in range(len(s) - 1))
        if double_digits and non_decreasing:
            yield x

def ilen(it):
    # Simple to understand:
    return sum(1 for x in it)
    # There is another more complicated solution:
    # https://stackoverflow.com/q/5384570

def day4():
    return ilen(day4bruteforce())

def day4b():
    return ilen(day4b_bruteforce())

day4(), day4b()

(1686, 1145)

# Day 5: Sunny with a Chance of Asteroids

<https://adventofcode.com/2019/day/5>

The CPU emulator now has a few more instructions:

* I/O: `in`, `out`
* Jumps: `jnz`, `jz`
* Comparisons: `lt`, `eq`

It also introduces the *immediate* addressing mode (in addition to the previously implemented *memory* (or *position*) addressing mode).

In [201]:
class day5Exception(Exception):
    pass


class day5Param:
    def __init__(self, cpu, value, mode):
        self.cpu = cpu
        self.value = value
        self.mode = mode

    def __repr__(self):
        return "day5Param({}, {}, {})".format(self.cpu, self.value, self.mode)

    def read(self):
        if self.mode == 0:
            # Position mode.
            return self.cpu.mem[self.value]
        elif self.mode == 1:
            # Immediate mode.
            return self.value
        elif self.mode == 2:
            # Relative mode, introduced in day 9.
            return self.cpu.mem[self.cpu.relative_base + self.value]
        else:
            raise day5Exception("Unsupported mode {}".format(self.mode))

    def write(self, x):
        if self.mode == 0:
            # Position mode.
            self.cpu.mem[self.value] = x
        elif self.mode == 1:
            # Immediate mode.
            raise day5Exception("Immediate mode cannot be written to")
        elif self.mode == 2:
            # Relative mode, introduced in day 9.
            self.cpu.mem[self.cpu.relative_base + self.value] = x
        else:
            raise day5Exception("Unsupported mode {}".format(self.mode))

            
class day5Computer:
    instructions = {
        i.opcode: i
        for i in [
            day2Instuction(1, "add", 4),
            day2Instuction(2, "mul", 4),
            day2Instuction(3, "in", 2),
            day2Instuction(4, "out", 2),
            day2Instuction(5, "jnz", 3),
            day2Instuction(6, "jz", 3),
            day2Instuction(7, "lt", 4),
            day2Instuction(8, "eq", 4),
            day2Instuction(9, "rel", 2),  # Introduced in day 9
            day2Instuction(99, "halt", 1),
        ]
    }

    def __init__(self, mem):
        self.mem = mem
        self.pc = 0  # Program Counter, AKA Instruction Pointer
        self.relative_base = 0  # Introduced in day 9
        self.halted = False
        self.input_cb = None
        #self.input_gen = None
        self.output_cb = None

    def __repr__(self):
        return '<day5Computer: mem_length={}, pc={}, halted={}>'.format(
            len(self.mem),
            self.pc,
            self.halted
        )

    def decode(self, code):
        opcode = code % 100        
        if opcode not in self.instructions:
            raise day5Exception("Invalid opcode {} at position {}".format(opcode, self.pc))

        instruction = self.instructions[opcode]

        modes = []
        modenumber = code // 100
        for x in range(1, instruction.length):
            modes.append(modenumber % 10)
            modenumber = modenumber // 10

        return instruction, modes

    def step(self):
        """Executes one instruction"""
        code = self.mem[self.pc]
        instruction, modes = self.decode(code)

        # Single-line that works for list-like memory (i.e. supports slices).
        #params = self.mem[self.pc+1:self.pc+instruction.length]

        # Alternative version that works on both lists and dicts.
        params = [
            self.mem[addr]
            for addr in range(self.pc+1, self.pc+instruction.length)
        ]

        if len(params) != len(modes):
            # Sanity check, may happen if we try to read beyond the end of the memory.
            raise day5Exception("Got {} params for pc={}, {}".format(len(params), self.pc, instruction))

        #print("pc={}->{} {} params={} modes={}".format(self.pc, code, instruction, params, modes))
        self.pc += instruction.length

        param_objs = [day5Param(self, *pm) for pm in zip(params, modes)]
        getattr(self, 'execute_' + instruction.name)(*param_objs)

    def step_if_not_halted(self):
        if not self.halted:
            self.step()

    def run(self, limit=None):
        while not self.halted and (limit is None or limit > 0):
            if limit is not None:
                limit -=1
            self.step()

    def execute_add(self, a, b, dst):
        dst.write(a.read() + b.read())
    def execute_mul(self, a, b, dst):
        dst.write(a.read() * b.read())
    def execute_in(self, x):
        if not callable(self.input_cb):
            raise day5Exception("Please set cpu.input_cb to a zero-argument function that retuns an integer")
        x.write(self.input_cb())

        #if self.input_gen:
        #    x.write(next(self.input_gen))
        #elif callable(self.input_cb):
        #    x.write(self.input_cb())
        #else:
        #    raise day5Exception("Please set cpu.input_cb to a zero-argument function that retuns an integer, or cpu.input_gen to a generator")
    def execute_out(self, x):
        if not callable(self.output_cb):
            raise day5Exception("Please set cpu.output_cb to a one-argument function")
        self.output_cb(x.read())
    def execute_jnz(self, x, dst):
        if x.read() != 0:
            self.pc = dst.read()
    def execute_jz(self, x, dst):
        if x.read() == 0:
            self.pc = dst.read()
    def execute_lt(self, a, b, dst):
        dst.write(1 if a.read() < b.read() else 0)
    def execute_eq(self, a, b, dst):
        dst.write(1 if a.read() == b.read() else 0)
    def execute_rel(self, base):
        # Introduced in day 9.
        self.relative_base += base.read()
    def execute_halt(self):
        self.halted = True

In [60]:
def day5input():
    with open("day5input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

def day5(inputs=None):
    # echo program, reads and then writes.
    #mem = [3,0, 4,0, 99]
    
    # Prints 999, 1000 or 1001, after comparing the input with 8.
    #mem = [
    #    3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
    #    1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
    #    999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
    #]

    # TEST - Thermal Environment Supervision Terminal
    mem = day5input()

    computer = day5Computer(mem)
    if inputs is None:
        # Interactive mode.
        computer.input_cb = lambda: int(input())
    else:
        computer.input_cb = iter(inputs).__next__
    computer.output_cb = print
    computer.run(limit=9999)

print("Input = 1")
day5([1])
print("Input = 5")
day5([5])

Input = 1
0
0
0
0
0
0
0
0
0
13933662
Input = 5
2369720


# Day 6: Universal Orbit Map

<https://adventofcode.com/2019/day/6>

A graph that is a actually tree, which greatly simplifies the algorithms.

In [17]:
def day6sample():
    yield from (line.strip().split(")") for line in """
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
    """.strip().splitlines())

def day6input():
    with open("day6input.txt") as f:
        for line in f.readlines():
            if line.strip():
                yield line.strip().split(')')

class day6Node:
    def __init__(self, name, *, parent=None, distance=None):
        self.name = name
        self.distance = parent
        self.parent = distance

    def __repr__(self):
        return "<day6Node {} distance={} parent={}>".format(self.name, self.distance, self.parent)
    def __str__(self):
        return self.name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        return self.name == self.other

    def calc_distance(self):
        if self.distance is None:
            if self.parent is None:
                self.distance = 0
            else:
                self.distance = self.parent.calc_distance() + 1
        return self.distance

    def path_to_root(self):
        if self.parent is None:
            return
        yield self.parent
        yield from self.parent.path_to_root()

class defaultdict_with_arg(dict):
    """Variation of defauldict that passes the missing key as argument to the factory
    
    https://stackoverflow.com/q/25951966/124946
    """
    def __init__(self, default_factory):
        self.default_factory = default_factory
    def __missing__(self, key):
        res = self[key] = self.default_factory(key)
        return res

def day6build_graph(pairs):
    g = defaultdict_with_arg(lambda name: day6Node(name))
    for parent, child in pairs:
        g[child].parent = g[parent]
    return g

def day6sampledebug():
    g = day6build_graph(day6sample())
    for name, node in sorted(g.items()):
        node.calc_distance()
        print(repr(node))
    print(sum(node.calc_distance() for node in g.values()))

def day6():
    g = day6build_graph(day6input())
    return sum(node.calc_distance() for node in g.values())

def day6b():
    g = day6build_graph(day6input())
    #print('-'.join(str(node) for node in g["YOU"].path_to_root()))
    #print('-'.join(str(node) for node in g["SAN"].path_to_root()))
    p1 = set(g["YOU"].path_to_root())
    p2 = set(g["SAN"].path_to_root())
    # symmetric_difference, returns a set of elements that are in only one of the sets.
    return len(p1 ^ p2)

day6(), day6b()

(106065, 253)

# Day 7: Amplification Circuit

<https://adventofcode.com/2019/day/7>

This problem requires running five CPUs in parallel, using input/output to transfer messages between them.

Go-routines with channels would have been quite handy here.

In [18]:
def day7input():
    with open("day7input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

class day7Amp:
    #program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
    program = day7input()
    #program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

    def __init__(self, phase):
        self.memory = self.program[:]
        self.ibuf = deque([phase])
        self.obuf = deque()
        self.computer = day5Computer(self.memory)
        self.computer.input_cb = self.input_cb
        self.computer.output_cb = self.output_cb

    def input_cb(self):
        return self.ibuf.popleft()
    def output_cb(self, value):
        self.obuf.append(value)
    def run(self):
        self.computer.run(limit=9999)
        assert(len(self.obuf) == 1)
        return self.obuf[0]

    def add_input(self, value):
        self.ibuf.append(value)
    def run_with_output_generator(self):
        while not self.computer.halted:
            while len(self.obuf) > 0:
                yield self.obuf.popleft()
            self.computer.step()

def day7build_and_run(phases):
    signal = 0
    for phase in phases:
        amp = day7Amp(phase=phase)
        amp.add_input(signal)
        signal = amp.run()
    return signal

def day7():
    #return day7build_and_run([4,3,2,1,0])
    return max(
        day7build_and_run(phases)
        for phases in
        #itertools.product(range(5), repeat=5)
        itertools.permutations(range(5))
    )

day7()

22012

In [19]:
def day7b_build_and_run(phases):
    amps = [day7Amp(phase=phase) for phase in phases]
    amps[0].add_input(0)
    for amp in amps:
        amp.gen = amp.run_with_output_generator()

    last_signal = 0
    current_amp = 0
    try:
        while True:
            signal = next(amps[current_amp].gen)
            current_amp += 1
            if current_amp >= len(amps):
                last_signal = signal
                current_amp = 0
            amps[current_amp].add_input(signal)
    except StopIteration:
        pass

    return last_signal

def day7b():
    #return day7b_build_and_run([9,8,7,6,5])
    return max(
        day7b_build_and_run(phases)
        for phases in
        #itertools.product(range(5), repeat=5)
        itertools.permutations(range(5, 10))
    )

day7b()

4039164

# Day 8: Space Image Format

<https://adventofcode.com/2019/day/8>

Rendering this highly inneficient image format, composed of several layers and a palette of three colors: black, white, transparent.

In [26]:
def day8input():
    with open("day8input.txt") as f:
        return [int(c) for c in f.read() if c.isdigit()]

def chunks(size, lst):
    """Yields chunks of size.
    Inspired by (or copied from) https://stackoverflow.com/q/312443
    """
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

def day8():
    data = day8input()
    w, h = 25, 6
    layer_counts = [Counter(layer) for layer in chunks(w * h, data)]
    # Note: the provided input doesn't have any layer with zero zeroes.
    fewest_zeros = min(c[0] for c in layer_counts)
    return [
        "Layer {}: {} 0s, {} 1s, {} 2s; 1s*2s={}".format(
            i,
            c[0],
            c[1],
            c[2],
            c[1] * c[2]
        )
        for (i, c) in enumerate(layer_counts)
        if c[0] == fewest_zeros
    ]

day8()

['Layer 5: 8 0s, 12 1s, 130 2s; 1s*2s=1560']

In [30]:
def day8b():
    data = day8input()
    w, h = 25, 6
    BLACK, WHITE, TRANS = 0, 1, 2
    CHARS = "█ ░"
    image = [TRANS] * (w * h)
    for layer in chunks(w * h, data):
        for i in range(w * h):
            if image[i] == TRANS:
                image[i] = layer[i]
    return "\n".join(
        "".join(CHARS[px] for px in row)
        for row in chunks(w, image)
    )

print(day8b())

 ██ ██  ███  ██ ██ █ ██ █
 ██ █ ██ █ ██ █ ██ █ ██ █
 ██ █ ████ ████ ██ █    █
 ██ █ █  █ ████ ██ █ ██ █
 ██ █ ██ █ ██ █ ██ █ ██ █
█  ███   ██  ███  ██ ██ █


# Day 9: Sensor Boost

<https://adventofcode.com/2019/day/9>

Adding one extra instruction and one new addressing mode to the Intcode computer from day 5. In fact, the changes were so minor that I just edited the Day 5 code.

In [109]:
def day9input():
    with open("day9input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

def day9():
    programs = [
        [
            "Quine",
            [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99],
            [],
        ],
        [
            "Outputs a 16-bit number",
            [1102,34915192,34915192,7,4,7,99,0],
            [],
        ],
        [
            
            "Outputs a large number",
            [104,1125899906842624,99],
            [],
        ],
        [
            "BOOST - test mode",
            day9input(),
            [1],
        ],
        [
            "BOOST - coordinates",
            day9input(),
            [2],
        ],
    ]

    for (name, mem, inputs) in programs:
        print(name)
        flexmem = defaultdict(lambda: 0, enumerate(mem))
        computer = day5Computer(flexmem)
        computer.input_cb = iter(inputs).__next__
        computer.output_cb = print
        computer.run(limit=999999)

day9()

Quine
109
1
204
-1
1001
100
1
100
1008
100
16
101
1006
101
0
99
Outputs a 16-bit number
1219070632396864
Outputs a large number
1125899906842624
BOOST - test mode
4261108180
BOOST - coordinates
77944


# Day 10: Monitoring Station

<https://adventofcode.com/2019/day/10>

Given a grid of (infinitely small) points, trace rays and find which points are obscured by other points; and also order the points by the angle of the rays.

In [57]:
def day10input():
    with open("day10input.txt") as f:
        return [line.strip() for line in f.read().strip().splitlines()]

def day10count_visible(x, y, chart):
    rays = set()
    for (ay, line) in enumerate(chart):
        for (ax, c) in enumerate(line):
            if c == "#":
                dx = ax - x
                dy = ay - y
                if dx != 0 or dy != 0:
                    gcd = math.gcd(dx, dy)
                    dx /= gcd
                    dy /= gcd
                    rays.add((dx, dy))
    return len(rays)

def day10find_best(chart):
    scores = [
        [
            (
                day10count_visible(x, y, chart)
                if c == "#"  # Only count if this position is an asteroid.
                else 0
            )
            for (x, c) in enumerate(line)
        ]
        for (y, line) in enumerate(chart)
    ]
    most = max(itertools.chain.from_iterable(scores))
    return [
        (x, y, c)
        for (y, line) in enumerate(scores)
        for (x, c) in enumerate(line)
        if c == most
    ]

def day10():
        # Sample input:
    chart = [
        line.strip() for line in
"""
.#..#
.....
#####
....#
...##
""".strip().splitlines()
    ]
    
    # Real input:
    chart = day10input()
    return day10find_best(chart)

day10()

[(11, 13, 227)]

In [82]:
# Testing and understanding atan2, angles, and vector rotations.
[
    (
        x,
        y,
        -math.degrees(
            math.atan2(x, -y)
        ),
        -math.atan2(x, -y),
    )
    for (x, y) in [
        (0, 0),

        (-1, 1),  # Should be the largest value.
        (-1, 0),
        (-1, -1),
        (0, -1),
        (1, -1),
        (1, 0),
        (1, 1),
        (0, 1),  # Should be the smallest value.
    ]
]

[(0, 0, -0.0, -0.0),
 (-1, 1, 135.0, 2.356194490192345),
 (-1, 0, 90.0, 1.5707963267948966),
 (-1, -1, 45.0, 0.7853981633974483),
 (0, -1, -0.0, -0.0),
 (1, -1, -45.0, -0.7853981633974483),
 (1, 0, -90.0, -1.5707963267948966),
 (1, 1, -135.0, -2.356194490192345),
 (0, 1, -180.0, -3.141592653589793)]

In [99]:
Day10Asteroid = namedtuple("Day10Asteroid", ["angle", "distance", "x", "y"])

def day10explore(x, y, chart):
    mapping = defaultdict(list)
    for (ay, line) in enumerate(chart):
        for (ax, c) in enumerate(line):
            if c == "#":
                dx = ax - x
                # Negating dy because in geometry y should increase upwards,
                # but in the data structure it increases downwards.
                dy = -(ay - y)
                if dx != 0 or dy != 0:
                    dist = abs(dx) + abs(dy)
                    gcd = math.gcd(dx, dy)
                    dx /= gcd
                    dy /= gcd
                    # atan2(y, x) returns the angle to +X axis.
                    # I'm rotating this vector 90deg because I want the angle to +Y.
                    angle = -math.atan2(dx, -dy)
                    # Note: It is usually a bad idea to use floating point as a key.
                    # However, the angle is derived from predictable integers, so the
                    # same floating point angle should be generated for aligned asteroids.
                    mapping[angle].append(Day10Asteroid(angle, dist, ax, ay))
    for lst in mapping.values():
        lst.sort()
    return mapping

def day10b():
    # Sample input:
    chart1 = [
        line.strip() for line in
"""
.#..#
.....
#####
....#
...##
""".strip().splitlines()
    ]

    # Sample input:
    chart2 = [
        line.strip() for line in
"""
..#..
.#.#.
#.#.#
.#.#.
..#..
""".strip().splitlines()
    ]

    # Real input:
    chart = day10input()

    ((x, y, _),) = day10find_best(chart)
    print("x={} y={}".format(x, y))

    ex = day10explore(x, y, chart)
    #pprint.pprint(ex)

    # For debugging.
    if False:
        # Converting list of strings to a list of list of chars.
        # i.e. making the chart mutable.
        debugchart = [
            [c for c in line]
            for line in chart
        ]
        chars = string.digits + string.ascii_letters
        cnt = 0
        for (angle, lst) in sorted(ex.items()):
            for ast in lst:
                debugchart[ast.y][ast.x] = chars[cnt]
                cnt += 1
        print("\n".join("".join(line) for line in debugchart))

    # Giant laser starts obliterating asteroids!
    cnt = 0
    boom = True
    asteroids_per_angle = sorted(ex.items())
    while boom:
        boom = False
        for (angle, lst) in asteroids_per_angle:
            if len(lst) > 0:
                boom = True
                ast = lst.pop(0)
                cnt += 1
                if False or cnt == 200:
                    print("#{:3}: {:= 4.0f}deg @ {:2},{:2}".format(
                        cnt, math.degrees(ast.angle), ast.x, ast.y
                    ))

day10b()

x=11 y=13
#200:  151deg @  6, 4


# Day 11: Space Police

<https://adventofcode.com/2019/day/11>

Implement a painting robot that is controlled by the Intcode computer (from days 5 and 9).

In [107]:
def day11input():
    with open("day11input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

class day11PaintingRobot:
    DIRS = [
        ( 0, -1),  # Up
        ( 1,  0),  # Right
        ( 0,  1),  # Down
        (-1,  0),  # Left
    ]
    def __init__(self):
        self.x = 0
        self.y = 0
        self.dir = 0
        self.canvas = dict()
        self.next_action = 0

    def camera(self):
        return self.canvas.get((self.x, self.y), 0)

    def action(self, value):
        if self.next_action == 0:
            self.paint(value)
        elif self.next_action == 1:
            self.turn(value)
            self.move()

        self.next_action = (self.next_action + 1) % 2

    def move(self):
        self.x += self.DIRS[self.dir][0]
        self.y += self.DIRS[self.dir][1]

    def paint(self, color):
        self.canvas[(self.x, self.y)] = color

    def turn(self, value):
        """Turns left if value is 0, turns right if value is 1.
        Undefined if value is not 0 nor 1.
        """
        self.dir = (self.dir - 1 + (2 * value) + len(self.DIRS)) % len(self.DIRS)

    def canvas_as_string(self):
        min_x = min(p[0] for p in self.canvas.keys())
        min_y = min(p[1] for p in self.canvas.keys())
        max_x = max(p[0] for p in self.canvas.keys())
        max_y = max(p[1] for p in self.canvas.keys())
        width = max_x - min_x + 1
        height = max_y - min_y + 1
        CHARS = "█ "
        # List of List of String
        lls = [
            [CHARS[0]] * width
            for i in range(height)
        ]
        for ((x, y), color) in self.canvas.items():
            lls[y - min_y][x - min_x] = CHARS[color]
        return "\n".join("".join(line) for line in lls)

def day11(b=False):
    mem = day11input()
    flexmem = defaultdict(lambda: 0, enumerate(mem))
    bot = day11PaintingRobot()
    if b:
        bot.canvas[(0, 0)] = 1
    computer = day5Computer(flexmem)
    computer.input_cb = bot.camera
    computer.output_cb = bot.action
    computer.run(limit=999999)
    print("Painted panels:", len(bot.canvas))
    return bot.canvas_as_string()

print(day11())
print(day11(b=True))

Painted panels: 2129
███████████████████████████████  █████████████████████████████████████████████████
███████████████████████████████ █   ██████████████████████████████████████████████
██████████████████████████████ █  █ ██████████████████████████████████████████████
████████████████████████████ ██ █ █ ██████████████████████████████████████████████
████████████████████████████       █ █████████████████████████████████████████████
████████████████████████    █ █ ███  █████████████████████████████████████████████
████████████████████████ ██  █████ ███████████████████████████████████████████████
███████████████████████ █  █ ██  █  ██████████████████████████████████████████████
████████████████  ██  █    ███████████████████████████████████████████████████████
█████████████   █ ██    ██  ████ ██ █  ███████████████████████████████████████████
██████████████ █ █  █ ████   █ █       ███████████████████████████████████████████
█████████████  █ █    █ ███ ███    ██ ████████████████████████████

# Day 12: The N-Body Problem

<https://adventofcode.com/2019/day/12>

Simulating gravity, velocity and position for four moons in space, using broken physics and incorrect definitions of energy.

In [142]:
class Vector:
    __slots__ = ["D", "coords"]

    def __init__(self, dimensions, coords):
        self.D = dimensions
        self.coords = [*coords]

    def __repr__(self):
        return "Vector({}, {})".format(self.D, self.coords)

    def __str__(self):
        return "<{}>".format(", ".join(str(v) for v in self.coords))

    def range(self):
        return range(self.D)

    @property
    def x(self):
        return self.coords[0]
    @property
    def y(self):
        return self.coords[1]
    @property
    def z(self):
        return self.coords[2]
    @property
    def w(self):
        return self.coords[3]

    def __eq__(self, other):
        return self.D == other.D and self.coords == other.coords
    def __hash__(self):
        return hash((self.D, *self.coords))

    def __len__(self):
        return self.D
    def __getitem__(self, key):
        return self.coords[key]
    def __iter__(self):
        return iter(self.coords)
    def __reversed__(self):
        return reversed(self.coords)

    def __pos__(self):
        return self
    def __neg__(self):
        return Vector(self.D, (-v for v in self.coords))
    def __abs__(self):
        return sum(v**2 for v in self.coords) ** (1 / self.D)

    def __add__(self, other):
        if isinstance(other, (int, float)):
            return Vector(self.D, (self.coords[i] + other for i in self.range()))
        else:
            return Vector(self.D, (self.coords[i] + other.coords[i] for i in self.range()))

    def __sub__(self, other):
        if isinstance(other, (int, float)):
            return Vector(self.D, (self.coords[i] - other for i in self.range()))
        else:
            return Vector(self.D, (self.coords[i] - other.coords[i] for i in self.range()))

    def __mul__(self, other):
        return Vector(self.D, (self.coords[i] * other for i in self.range()))

    def __truediv__(self, other):
        return Vector(self.D, (self.coords[i] / other for i in self.range()))

    def __radd__(self, other):
        return self + other
    def __rsub__(self, other):
        return self - other
    def __rmul__(self, other):
        return self * other
    def __rtruediv__(self, other):
        return self / other


class Vec2(Vector):
    def __init__(self, x, y):
        super().__init__(2, (x, y))


class Vec3(Vector):
    def __init__(self, x, y, z):
        super().__init__(3, (x, y, z))


class Vec4(Vector):
    def __init__(self, x, y, z, w):
        super().__init__(4, (x, y, z, w))

In [145]:
def cmp(a, b):
    if a < b:
        return -1
    elif a > b:
        return 1
    else:
        return 0

In [160]:
def day12gravity(a, b):
    """Returns a vector such as, for each axis, the value is -1 or 0 or +1."""
    return Vector(len(a), (cmp(b[i], a[i]) for i in a.range()))

class day12Moon:
    def __init__(self, pos, vel=Vec3(0, 0, 0)):
        self.pos = pos
        self.vel = vel

    def __repr__(self):
        return "day12Moon(pos={!s}, vel={!s})".format(self.pos, self.vel)

def day12input():
    with open("day12input.txt") as f:
        # Okay, this is ugly and hard to read (and hard to debug),
        # given how deeply nested this expression is. But it is cool!
        return [
            Vec3(
                *(
                    int(v)
                    for v in re.match(
                        r"<x=(-?\d+),y=(-?\d+),z=(-?\d+)>",
                        re.sub(r"\s+", "", line)
                    ).groups()
                )
            )
            for line in f.read().strip().splitlines()
        ]

def day12simulate(moons):
    # Step 1: update their velocities.
    for target in moons:
        gravity = sum(
            day12gravity(target.pos, other.pos)
            for other in moons
            if target is not other
        )
        target.vel += gravity

    # Step 2: update their positions.
    for target in moons:
        target.pos += target.vel

def day12():
    # Sample input:
    moons = [
        day12Moon(vec) for vec in [
            Vec3(x=-1, y=0, z=2),
            Vec3(x=2, y=-10, z=-7),
            Vec3(x=4, y=-8, z=8),
            Vec3(x=3, y=5, z=-1),
        ]
    ]
    moons = [day12Moon(vec) for vec in day12input()]

    for i in range(1000):
        #pprint.pprint(moons)
        day12simulate(moons)

    return sum(
        operator.mul(
            *(
                sum(abs(v) for v in vec) for vec in [moon.pos, moon.vel]
            )
        )
        for moon in moons
    )

day12()

9139

In [168]:
# Oh, so we need to be more efficient, huh?
# It turns out (after a quick look online) that, in this problem,
# each dimension (axis) is completely independent from the others.
# This is counter-intuitive to the real gravity force, but whatever.

def day12simulate1d(moons):
    # Step 1: update their velocities.
    for target in moons:
        gravity = sum(
            cmp(other[0], target[0])
            for other in moons
            if target is not other
        )
        target[1] += gravity

    # Step 2: update their positions.
    for target in moons:
        target[0] += target[1]

def day12tuplify(moons):
    # Receives a list of lists of two integer.
    # Returns a tuple.
    return tuple(itertools.chain.from_iterable(moons))

def lcm(a, b):
    return a * b // math.gcd(a, b)

def day12b():
    # Sample input:
    moons = [
        day12Moon(vec) for vec in [
            Vec3(x=-1, y=0, z=2),
            Vec3(x=2, y=-10, z=-7),
            Vec3(x=4, y=-8, z=8),
            Vec3(x=3, y=5, z=-1),
        ]
    ]
    moons = [day12Moon(vec) for vec in day12input()]

    dimensions = [
        [
            [moon.pos[d], moon.vel[d]]
            for moon in moons
        ]
        for d in range(3)
    ]
    cycle_counts = []
    for axis in dimensions:
        cnt = 0
        past = dict()
        past[day12tuplify(axis)] = cnt
        initial = None
        cycle = None
        while True:
            day12simulate1d(axis)
            cnt += 1
            t = day12tuplify(axis)
            if t in past:
                initial = past[t]
                cycle = cnt - past[t]
                break
            else:
                past[t] = cnt
        print(initial, cycle, cnt)
        # Thankfully, all cycles go back to the initial position.
        # In other words, initial is always zero for the provided inputs.
        cycle_counts.append(cycle)
    return functools.reduce(lcm, cycle_counts)

day12b()

0 268296 268296
0 231614 231614
0 108344 108344


420788524631496

# Day 13: Care Package

<https://adventofcode.com/2019/day/13>

Build a screen capable of displaying a grid. The problem description does not specify the screen dimensions. I'm just reusing the code from the painting robot (from day 11) and adapting to this problem.

In [118]:
def day13input():
    with open("day13input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

class day13Screen:
    PALETTE = [
        " ",  # 0 is an empty tile. No game object appears in this tile.
        "█",  # 1 is a wall tile. Walls are indestructible barriers.
        "□",  #"▒",  # 2 is a block tile. Blocks can be broken by the ball.
        "═",  # 3 is a horizontal paddle tile. The paddle is indestructible.
        "●",  # 4 is a ball tile. The ball moves diagonally and bounces off objects.
    ]
    def __init__(self):
        self.x = 0
        self.y = 0
        self.next_action = 0
        self.canvas = dict()
        self.score = 0
        # Cheating (or auto-play).
        self.paddle_x = 0
        self.ball_x = 0

    def action(self, value):
        if self.next_action == 0:
            self.x = value
        elif self.next_action == 1:
            self.y = value
        elif self.next_action == 2:
            if self.x == -1 and self.y == 0:
                self.score = value
            else:
                self.canvas[(self.x, self.y)] = value
                if value == 3:
                    self.paddle_x = self.x
                elif value == 4:
                    self.ball_x = self.x

        self.next_action = (self.next_action + 1) % 3

    def canvas_as_string(self):
        min_x = min(p[0] for p in self.canvas.keys())
        min_y = min(p[1] for p in self.canvas.keys())
        max_x = max(p[0] for p in self.canvas.keys())
        max_y = max(p[1] for p in self.canvas.keys())
        width = max_x - min_x + 1
        height = max_y - min_y + 1
        # List of List of String
        lls = [
            [self.PALETTE[0]] * width
            for i in range(height)
        ]
        for ((x, y), color) in self.canvas.items():
            lls[y - min_y][x - min_x] = self.PALETTE[color]
        return "{}\n".format(self.score) + "\n".join("".join(line) for line in lls)

def day13():
    mem = day13input()
    flexmem = defaultdict(lambda: 0, enumerate(mem))
    screen = day13Screen()
    computer = day5Computer(flexmem)
    computer.input_cb = input
    computer.output_cb = screen.action
    computer.run(limit=999999)
    print("Blocks: {}".format(
        sum(
            1
            for tile in screen.canvas.values()
            if tile == 2
        )
    ))
    return screen.canvas_as_string()

print(day13())

Blocks: 355
0
██████████████████████████████████████████
█                                        █
█ □□ □ □□ □□ □□□□□□ □□□ □ □□ □□ □□□  □□  █
█ □□□   □  □□ □ □ □□  □□ □□□ □ □□□□ □□ □ █
█ □□□□□    □ □□□ □□   □  □  □ □□  □  □ □ █
█ □  □□□□□□ □ □□    □  □□□ □    □□□□ □ □ █
█  □ □ □   □□□□□□□ □□□ □□   □ □□ □   □□□ █
█ □□    □   □□   □□□□□  □□□ □□□□ □□ □  □ █
█ □ □□ □□ □ □□ □ □□  □ □  □□ □□□□□□  □□  █
█   □□□ □□□□ □□ □□□     □□ □□□□□□  □□□□□ █
█ □  □□□□   □ □ □□□□□□□ □ □  □   □ □□□   █
█ □  □□ □ □ □□  □ □     □□□  □  □□ □□ □□ █
█      □ □□□□□ □□ □ □□ □  □□□□□    □□  □ █
█   □  □□ □□ □□ □□   □ □  □□□ □ □ □□□□□□ █
█  □□□□□□  □ □  □ □ □□□ □ □□□  □□ □ □□□□ █
█    □ □□□□□ □□ □ □ □□ □□ □□ □□□ □□□  □□ █
█ □  □ □ □       □ □ □  □□□□ □ □□ □  □□□ █
█  □ □□□□□ □□□□□□□ □□□□ □   □□ □□□□  □□□ █
█                                        █
█                  ●                     █
█                                        █
█                                        █
█                    ═                  

In [147]:
def day13b():
    mem = day13input()
    mem[0] = 2  # Free play! No coins needed!
    flexmem = defaultdict(lambda: 0, enumerate(mem))
    screen = day13Screen()

    # Auto-play AI:
    #player = lambda: 0 if screen.ball_x == screen.paddle_x else 1 if screen.ball_x > screen.paddle_x else -1
    player = lambda: cmp(screen.ball_x, screen.paddle_x)

    computer = day5Computer(flexmem)
    computer.input_cb = player
    computer.output_cb = screen.action
    computer.run(limit=999999)
    return screen.canvas_as_string()

print(day13b())

18371
██████████████████████████████████████████
█                                        █
█                                     ●  █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                        █
█                                     ═  █
█    

# Day 14: Space Stoichiometry

<https://adventofcode.com/2019/day/14>

Sounds like a simple problem of simulating reactions through a simple dependency graph. Until we get to part two, which requires a gigantic amount of reactions, making me go back to the original code and optimize it by adding the `multiplier` parameter.

In [187]:
# This would have been easier with Python 3.7 dataclass, or attrs module.
# But right now I have Python 3.6 and I don't want to use stuff outside the standard library.
#day14thing = namedtuple("day14thing", ["qty", "name"])

# https://stackoverflow.com/q/14822184
def ceildiv(a, b):
    return -(-a // b)

class day14Thing:
    """One portion of a reaction. Encapsulates a name and a quantity.
    """
    def __init__(self, qty=None, name=None, string=None):
        if string is not None:
            qty, name = string.strip().split()
        self.qty = int(qty)
        self.name = name.strip()
        # List of qty+names needed for this.
        self.deps = []
        # How many do we have?
        self.inventory = 0
        # How many have we produced in total.
        self.produced = 0

    def __repr__(self):
        return "day14Thing({}, {}, <{} deps>, <inventory={}>, <produced={}>)".format(
            self.qty,
            self.name,
            len(self.deps),
            self.inventory,
            self.produced,
        )
    def __str__(self):
        return "<{} {}>".format(self.qty, self.name)

    def produce_by_reaction(self, graph, multiplier):
        for dep in self.deps:
            graph[dep.name].consume(graph, qty=dep.qty * multiplier)
        self.inventory += self.qty * multiplier
        self.produced += self.qty * multiplier

    def consume(self, graph, qty=1):
        # Consuming our inventory.
        from_inv = min(self.inventory, qty)
        self.inventory -= from_inv
        qty -= from_inv

        # Producing the lowest multiple greater than qty.
        how_many = ceildiv(qty, self.qty)
        self.produce_by_reaction(graph, how_many)

        self.inventory -= qty


def day14parse(s):
    elements = dict()
    for line in s.strip().splitlines():
        inputs, _, outputs = line.strip().partition("=>")
        reactants = [
            day14Thing(string=thing)
            for thing in inputs.strip().split(",")
        ]
        product = day14Thing(string=outputs)
        product.deps = reactants
        elements[product.name] = product
    # Fake reaction for ORES.
    elements["ORE"] = day14Thing(1, "ORE")
    return elements

def day14input():
    with open("day14input.txt") as f:
        return f.read()

def day14():
    sample1 = """
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
    """
    sample2 = """
9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL
"""
    graph = day14parse(sample2)
    graph = day14parse(day14input())

    graph["FUEL"].consume(graph, 1)
    #pprint.pprint(graph)
    return graph["ORE"].produced

day14()

114125

In [189]:
def day14b_bruteforce():
    graph = day14parse(day14input())

    while graph["ORE"].produced <= 10000000: # 1000000000000:
        graph["FUEL"].consume(graph, 1)

    return graph["FUEL"].produced - 1

#day14b_bruteforce()

In [190]:
def day14b_cycle():
    graph = day14parse(day14input())    

    cycle = False
    while not cycle:
        graph["FUEL"].consume(graph, 1)
        cycle = all(
            el.inventory == 0
            for el in graph.values()
            if el.name != "FUEL"
        )

    pprint.pprint(graph["ORE"])
    pprint.pprint(graph["FUEL"])

    return graph["FUEL"].produced - 1

# Still not good, takes too long, there is no short cycle.
#day14b_cycle()

In [200]:
def day14b():
    graph = day14parse(day14input())

    graph["FUEL"].consume(graph, 1)
    # The amount of ORE consumed in the first run (when we have zero inventory of anything)
    # can be considered the upper bound of ORE to be consumed for each FUEL.
    ore_first_run = graph["ORE"].produced
    print("ore_first_run={}".format(ore_first_run))

    total_ore = 1000000000000

    while graph["ORE"].produced < total_ore:
        estimate = (total_ore - graph["ORE"].produced) // ore_first_run
        graph["FUEL"].consume(graph, max(estimate, 1))
        print(
            "estimate={:8} ore.produced={:13} ore.remaining={:13} fuel.produced={:8}".format(
                estimate,
                graph["ORE"].produced,
                total_ore - graph["ORE"].produced,
                graph["FUEL"].produced,
            )
        )

    return graph["FUEL"].produced - 1

day14b()

ore_first_run=114125
estimate= 8762321 ore.produced= 727803444969 ore.remaining= 272196555031 fuel.produced= 8762322
estimate= 2385073 ore.produced= 925908942368 ore.remaining=  74091057632 fuel.produced=11147395
estimate=  649209 ore.produced= 979832614466 ore.remaining=  20167385534 fuel.produced=11796604
estimate=  176713 ore.produced= 994510498903 ore.remaining=   5489501097 fuel.produced=11973317
estimate=   48100 ore.produced= 998505702302 ore.remaining=   1494297698 fuel.produced=12021417
estimate=   13093 ore.produced= 999593217860 ore.remaining=    406782140 fuel.produced=12034510
estimate=    3564 ore.produced= 999889241250 ore.remaining=    110758750 fuel.produced=12038074
estimate=     970 ore.produced= 999969819084 ore.remaining=     30180916 fuel.produced=12039044
estimate=     264 ore.produced= 999991737175 ore.remaining=      8262825 fuel.produced=12039308
estimate=      72 ore.produced= 999997717608 ore.remaining=      2282392 fuel.produced=12039380
estimate=      19 o

12039407

# Day 15: Oxygen System

<https://adventofcode.com/2019/day/15>

We have to build a program to brute-force the droid in order to explore all the area and figure out where are all the walls and where is our target (the oxygen system). I'm using a variation of day 13 screen. Then I have to calculate the minimum distance to the oxygen system, and then the maximum distance from the oxygen system to any point.

In [222]:
def day15input():
    with open("day15input.txt") as f:
        return [int(x) for x in f.read().strip().split(",")]

class day15Chart:
    PALETTE = [
        "?",  # 0 is unexplored
        "█",  # 1 is a wall
        " ",  # 2 is empty space
        "x",  # 3 is the oxygen system
        "D",  # 4 is the origin
    ]
    UNKNOWN = 0
    WALL    = 1
    EMPTY   = 2
    TARGET  = 3
    ORIGIN  = 4

    DIR = [
        ( 0,  0),  # nothing (0) - invalid input
        ( 0, -1),  # north (1)
        ( 0,  1),  # south (2)
        (-1,  0),  # west (3)
        ( 1,  0),  # east (4)
    ]
    def __init__(self, cpu):
        self.cpu = cpu
        cpu.input_cb = self.cpu_input
        cpu.output_cb = self.cpu_output
        self.cpu_next_input = None
        self.cpu_last_output = None

        self.x = 0
        self.y = 0
        self.canvas = defaultdict(lambda: 0, { (0, 0): self.ORIGIN })

    def xy(self, dir=0):
        return (
            self.x + self.DIR[dir][0],
            self.y + self.DIR[dir][1],
        )

    def look_around(self, recursive=False):
        for (d, undo) in [(1, 2), (2, 1), (3, 4), (4, 3)]:
            dest = self.xy(d)
            if self.canvas[dest] == self.UNKNOWN:
                has_moved = self.move(d)
                if has_moved:
                    if recursive:
                        self.look_around(recursive=recursive)
                    self.move(undo)

    def move(self, dir):
        self.cpu_next_input = dir
        while not self.cpu.halted and self.cpu_next_input is not None:
            self.cpu.step()

        was_wall = self.cpu_last_output == 0
        self.cpu_last_output = None

        # Has the droid moved?
        return not was_wall

    def cpu_input(self):
        return self.cpu_next_input
    def cpu_output(self, value):
        self.cpu_last_output = value
        origin = self.xy()
        dest = self.xy(self.cpu_next_input)
        unexplored = self.canvas[dest] == self.UNKNOWN
        if value == 0:
            # Wall
            if unexplored:
                self.canvas[dest] = self.WALL
        elif value == 1:
            # Empty
            if unexplored:
                self.canvas[dest] = self.EMPTY
            self.x, self.y = dest
        elif value == 2:
            # Target
            if unexplored:
                self.canvas[dest] = self.TARGET
            self.x, self.y = dest
        else:
            raise RuntimeError("Unexpected output value {!r}".format(value))
        #print(self.cpu_next_input, origin, "->", dest, value)
        self.cpu_next_input = None

    def breadth_first_search(self, start_pos):
        queue = deque([])
        queue.append((start_pos, 0))
        explored = set()
        explored.add(start_pos)
        most_dist = 0
        while len(queue) > 0:
            pos, dist = queue.popleft()
            most_dist = max(most_dist, dist)
            for d in [1, 2, 3, 4]:
                dest = (
                    pos[0] + self.DIR[d][0],
                    pos[1] + self.DIR[d][1],
                )
                dest_tile = self.canvas[dest]
                if dest_tile == self.TARGET:
                    print("Oxygen system at {}, {} moves away".format(dest, dist + 1))

                if dest_tile == self.WALL:
                    pass
                elif dest in explored:
                    pass
                else:
                    explored.add(dest)
                    queue.append((dest, dist + 1))
        print("Most moves needed = {}".format(most_dist))

    def canvas_as_string(self):
        min_x = min(p[0] for p in self.canvas.keys())
        min_y = min(p[1] for p in self.canvas.keys())
        max_x = max(p[0] for p in self.canvas.keys())
        max_y = max(p[1] for p in self.canvas.keys())
        width = max_x - min_x + 1
        height = max_y - min_y + 1
        # List of List of String
        lls = [
            [self.PALETTE[0]] * width
            for i in range(height)
        ]
        for ((x, y), color) in self.canvas.items():
            lls[y - min_y][x - min_x] = self.PALETTE[color]
        return "\n".join("".join(line) for line in lls)


def day15():
    mem = day15input()
    flexmem = defaultdict(lambda: 0, enumerate(mem))

    computer = day5Computer(flexmem)
    chart = day15Chart(computer)
    # Recursive look-around is essentially a flood-fill algorithm.
    chart.look_around(recursive=True)
    chart.breadth_first_search((0, 0))
    chart.breadth_first_search((-18, 12))
    return chart.canvas_as_string()

print(day15())

Oxygen system at (-18, 12), 374 moves away
Most moves needed = 390
Oxygen system at (-18, 12), 2 moves away
Most moves needed = 482
?█████?███████████?█████?███?███████████?
█     █           █     █   █           █
?██ █ ███████ ███ █ ███ █ █ ███████████ █
█   █ █     █ █   █ █   █ █         █   █
█ ███ █ ███ █ ███ █ █ ███ █████████ █ █ █
█ █ █ █ █   █   █   █ █     █       █ █ █
█ █ █ █ █ █████ █████ █ █████ ███████ █ █
█ █     █ █     █     █ █     █       █ █
█ ███████ █ █████ █████ █ █████ ███████ █
█       █   █   █   █   █   █   █       █
█ █████ █████ █ ███ █ █████ ███ █ ██████?
█     █ █ █   █   █ █ █     █   █ █     █
?██████ █ █ █ █████ █ █ █████ ███ █ ███ █
█   █   █ █ █       █ █     █ █ █ █ █   █
█ █ █ ███ █ █████████ █████ █ █ █ █ █ ██?
█ █   █   █ █   █   █   █   █ █   █ █   █
█ ███████ █ █ █ █ █████ █ ███ █ ███████ █
█   █   █   █ █   █     █ █   █ █     █ █
█ █ █ █ █ ███ █ ███ █████ ███ █ █ ███ █ █
█ █   █ █ █   █ █       █ █   █   █ █ █ █
?██████ █ █ ███ █ █████ █ █ 

# Day 16: Flawed Frequency Transmission

<https://adventofcode.com/2019/day/16>

In [4]:
def foo():
    for off in range(1, 10):
        print(" ".join(
            str(((i+1)//off) % 4) for i in range(20)
        ))
foo()

1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0
0 1 1 2 2 3 3 0 0 1 1 2 2 3 3 0 0 1 1 2
0 0 1 1 1 2 2 2 3 3 3 0 0 0 1 1 1 2 2 2
0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 0 0 0 0 1
0 0 0 0 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 0
0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3
0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2
0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2


In [26]:
def slurp(filename, binary=False):
    """Given a filename, reads it and returns it as a string."""
    with open(filename, "rb" if binary else "rt") as f:
        return f.read()

def day16fft_phase(signal, pattern):
    return [
        abs(sum(
            signal[i] * pattern[((i+1)//(pos+1)) % len(pattern)]
            for i in range(len(signal))
        )) % 10
        for pos in range(len(signal))
    ]

def day16():
    base_pattern = [0, 1, 0, -1]
    #s = "12345678"
    s = slurp("day16input.txt")
    signal = [int(c) for c in s.strip()]
    with time_this("100 iterations"):
        for phase in range(100):
            signal = day16fft_phase(signal, base_pattern)
    print("".join(str(v) for v in signal))

day16()

100 iterations took 9.972264s.
44098263577767243566465336017484725735409415599895796691021765237776874236590302055780078434486648834240933636976799566773394909867684152933521861174395404543180086836886119875299293558965440502143821973095956509317454599208750519403795800116220801205701913422524240286208878108663213257152190685157009953136173753150428016283102694406655192639305380491258511451386484958938572258565111520396204900638625168489737372772002077482523488722666436234360060362842815301561113791445180697878452550487283315061379956703398946235025295941452853075179024289281258573760516862974004335894617213438227837930586659611531495453514024517982663554177030764602168262


In [24]:
# Heavily inspired by https://stackoverflow.com/a/20924212
@contextmanager
def time_this(name="This code"):
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    print("{} took {:.6f}s.".format(name, end - start))

In [25]:
def day16b_naive():
    base_pattern = [0, 1, 0, -1]
    s = "1"
    #s = "03036732577212944063491565474664"
    #s = slurp("day16input.txt")
    s = s.strip()
    msg_offset = int(s[:7])
    signal = [int(c) for c in s] * 10000
    for phase in range(1):
        with time_this("One iteration for size {}".format(len(s))):
            signal = day16fft_phase(signal, base_pattern)
        print("".join(str(v) for v in signal[msg_offset:msg_offset+8]))

day16b_naive()

One iteration for size 1 took 26.959425s.
02006087
